In [ ]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [90]:
import sys
import time
import pandas as pd 
import selenium 
import re 
import requests 
import urllib.request 
from selenium import webdriver
from bs4 import BeautifulSoup 
from tqdm import tnrange 
from urllib.request import urlopen 
from tqdm import tqdm

In [91]:
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

In [92]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("disable-gpu")

In [93]:
driver = webdriver.Chrome('chromedriver', options=chrome_options)

In [94]:
driver.maximize_window()

In [95]:
# 한국어 문장을 영어, 일본어, 중국어로 번역합니다.
def kor_to_trans(text_data, trans_lang): 
    """ trans_lang에 넣는 파라미터 값: 'en' -> 영어 'ja&hn=0' -> 일본어 'zh-CN' -> 중국어(간체) """ 
    trans_list = []
    for i in tqdm(range(len(text_data))): 
        try: 
            driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+text_data[i])
            time.sleep(2.5)
            backtrans = driver.find_element_by_xpath('//*[@id="txtTarget"]').text
            
        except: 
            backtrans = text_data[i]

        if "XKZ" not in backtrans:
            backtrans = "XKZ" + backtrans
        if "KVX" not in backtrans:
            backtrans += "KVX"
        trans_list.append(backtrans)

    return trans_list
    

In [96]:
# 번역된 문장을 다시 한국어로 back translation 합니다
def trans_to_kor(transed_list, transed_lang):
    back_trans_list = []
    for i in tqdm(range(len(transed_list))): 
        try: 
            driver.get('https://papago.naver.com/?sk='+transed_lang+'&tk=ko&st='+transed_list[i])
            time.sleep(2.5) 
            backtrans = driver.find_element_by_xpath('//*[@id="txtTarget"]').text

        except: 
            backtrans = transed_list[i]    
        
        if "XKZ" not in backtrans:
            backtrans = "XKZ" + backtrans
        if "KVX" not in backtrans:
            backtrans += "KVX"
        back_trans_list.append(backtrans)
    return back_trans_list

In [97]:
# 고유명사를 **와 @@로 대치합니다
def proper_noun(df):
    for idx in range(len(df)):

        sub_entity = eval(df.iloc[idx, 2])
        sub_word = sub_entity['word']
        sub_s_idx = sub_entity['start_idx']
        sub_e_idx = sub_entity['end_idx']

        obj_entity = eval(df.iloc[idx, 3])
        obj_word = obj_entity['word']
        obj_s_idx = obj_entity['start_idx']
        obj_e_idx = obj_entity['end_idx']


        df.iloc[idx, 1] = df.iloc[idx, 1].replace(sub_word, "XKZ")
        df.iloc[idx, 1] = df.iloc[idx, 1].replace(obj_word, "KVX")


In [98]:
# **와 @@를 다시 원래 단어로 복원합니다
def recover_proper_noun(df):
    for idx in range(len(df)):
        sub_entity = eval(df.iloc[idx, 2])
        sub_word = sub_entity['word']
        
        obj_entity = eval(df.iloc[idx, 3])
        obj_word = obj_entity['word']
        
        df.iloc[idx, 1] = df.iloc[idx, 1].replace("XKZ", sub_word)
        df.iloc[idx, 1] = df.iloc[idx, 1].replace("KVX", obj_word)

        sub_s_idx = df.iloc[idx, 1].find(sub_word)
        sub_e_idx = sub_s_idx + len(sub_word) -1

        obj_s_idx = df.iloc[idx, 1].find(obj_word)
        obj_e_idx = obj_s_idx + len(obj_word) - 1

        df.iloc[idx, 2] = "{'word': "+str(sub_word)+", 'start_idx': "+str(sub_s_idx)+", 'end_idx': "+str(sub_e_idx)+"}"
        df.iloc[idx, 3] = "{'word': "+str(obj_word)+", 'start_idx': "+str(obj_s_idx)+", 'end_idx': "+str(obj_e_idx)+"}"

In [99]:
df = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/NLP/Korean_NLP/data/train.csv")
# 특정 수 이하의 문장을 가지는 라벨을 augmentation하기 위해 선택합니다
aug_label = []
for label in df['label'].unique():
    if (df['label'].value_counts() > 70)[label] == True and (df['label'].value_counts() <= 100)[label] == True:
        aug_label.append(label)

corpus = []
for idx in range(len(df)):
    if df.loc[idx, 'label'] in aug_label:
        corpus.append(df.loc[idx])
corpus_df = pd.DataFrame(corpus)


proper_noun(corpus_df)

In [100]:
len(corpus_df)

276

In [ ]:
for la, lang in zip(['eng', 'jpn', 'ch'], ['en', 'ja&hn=0', 'zh-CN']):
    transed_list = []
    backtransed_list = []
    exec(f'df_{la} = corpus_df.copy()')
    transed_list.extend(kor_to_trans(corpus_df['sentence'].values, lang))
    print(f"{la}언어로 번역된 문장: {transed_list}")
    backtransed_list.extend(trans_to_kor(transed_list, lang))
    exec(f'df_{la}["sentence"] = backtransed_list')
    print(f"{la}언어가 다시 한국어로 번역된 corpus: {backtransed_list}")

100%|██████████| 276/276 [13:12<00:00,  2.87s/it]


eng언어로 번역된 문장: ['XKZ entered KVX School and led the heyday of KVX School with Hwang Dong-il and Shin Young-seok, winning three gold medals: the Summer Games, the National Sports Festival, and the Strongest Competition.', "On November 26, 1924, the form of government was changed from the monarchy to the people's KVX, and the national title was designated as XKZ, becoming the second communist country in the world.", 'When he was a student at KVX School, he was a student of right-handed pitcher XKZ and admission, and the two were called "Jwa Won-sam" and "Yu Gi-pyo," respectively, at the time of joining the team.', 'This is because XKZ, a KVX theologian and professor at Union Theological Seminary, sent an invitation after setting up a position as a theology professor.', 'The inscription also mentions that XKZ is a Vishnufa KVX believer, and according to the inscription, XKZ performed various KVX ceremonies, including Maje Ashbameda, who was held when kings of the Vedas won a great victory

100%|██████████| 276/276 [13:10<00:00,  2.86s/it]


eng언어가 다시 한국어로 번역된 corpus: ['XKZ는 KVX스쿨에 입학하여 황동일, 신영석과 함께 KVX스쿨의 전성기를 이끌었으며, 하계 올림픽, 전국체전, 최강전 3관왕을 차지했다.', '1924년 11월 26일, 정부의 형태가 군주제에서 인민의 KVX로 바뀌었고, 국가 호칭은 XKZ로 지정되었고, 세계에서 두 번째 공산주의 국가가 되었다.', 'KVX스쿨 재학 시절 오른손 투수 XKZ와 입학 당시 좌원삼과 유기표로 불렸다.', 'KVX 신학자이자 유니온신학교 교수인 XKZ가 신학교수로 자리를 잡은 뒤 초청장을 보냈기 때문이다.', '그 비문에 의하면, XKZ는 비슈누파 KVX 신봉자이며, 비문에 따르면, 베다 왕들이 큰 승리를 거두었을 때 열렸던 Maje Ashbameda를 포함하여, 다양한 KVX 의식을 행했다.', '러일전쟁 때 XKZ가 1905년 동학의 전통에 따라 항일 사상을 담아 KVX를 전파하자 친일 사상으로 시천교를 세우고 종교지도자가 됐다.', 'XKZ의 통치자인 교황은 KVX 교회의 종교적 지도자이기도 했다.', '캄푸치아 인민공화국, 에티오피아 인민민주공화국, 남예멘은 KVX를 포기했고 XKZ에서 혁명이 일어나 KVX의 일당 체제가 붕괴되었다.', '폴란드는 20세기 초까지만 해도 교복의 개념이 존재하지 않았지만, 폴란드에서 KVX 정권 XKZ가 수립된 후 교복을 입었지만, 1980년대 경제 악화로 인해 많은 학교들이 교복을 취소했다.', '레베데프는 고르바초프 전 소련 대통령과도 협력하고 있으며 KVX 기반 신문인 XKZ의 지분 49%를 소유하고 있다.', '일본의 KVX는 XKZ 주홍법대사(홍법대사 774-835)가 중국 당나라 장안으로 이주했을 때 혜과로부터 배운 난해한 관계를 바탕으로 만들어졌다.', '2015년, KVX에서 공부하던 XKZ는 프랑스 영화 감독 장 클로드 부르랏이 공동 제작한 중국과 프랑스 영화 "개의 남자"에 주인공으로 캐스팅되었다.', '윌리엄 벤튼 스크랜턴(18561922) 선교사는 미국 북KVX협회가 

 26%|██▌       | 72/276 [03:27<09:50,  2.89s/it]

In [ ]:
for df in [df_eng, df_jpn, df_ch]:
    recover_proper_noun(df)

In [ ]:
df_integrated = pd.concat([df_eng, df_jpn, df_ch])

In [ ]:
file_name = "backtranslated_corpus_70_100.csv"

In [ ]:
df_integrated.to_csv(f"/content/drive/MyDrive/ColabNotebooks/NLP/Korean_NLP/back_translation_data/{file_name}")